In [9]:
from __future__ import print_function, unicode_literals, absolute_import, division
import sys
import numpy as np
import matplotlib
matplotlib.rcParams["image.interpolation"] = 'none'
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from glob import glob
from tifffile import imread
from tqdm import tqdm
from csbdeep.utils import Path, normalize
from csbdeep.io import save_tiff_imagej_compatible
import skimage as sk
import os

from stardist import random_label_cmap, _draw_polygons, export_imagej_rois
from stardist.models import StarDist2D

np.random.seed(6)
lbl_cmap = random_label_cmap()

In [24]:
def StarDist2D_with_stitching(imgs, axis_norm, model, save_path):
    print('Normalizing images')
    norm = [normalize(x, 1,99.8, axis=(0,)+tuple(1+np.array(axis_norm))) for x in tqdm(imgs)]
    print('Predicting')
    c=0
    for i in tqdm(norm):
        labels = [model.predict_instances(frame, return_labels=True)[0] for frame in i]
        labels = np.asarray(labels)
        labels = labels > 0
        labels = sk.measure.label(labels)
        sk.io.imsave(os.path.join(save_path,'labels_'+os.path.basename(Xfiles[c])+'.tiff'),labels,check_contrast=False)
        c+=1
        

In [11]:
Xfiles = sorted(glob('E:/Lempradl_Lab/April/OIC-78_Drosophila-brain-lesion-quant/training_images/*.tif'))
Ximgs = list(map(imread,Xfiles))
test_img = Ximgs[0]
n_channel = 1 if X[0].ndim == 2 else X[0].shape[-1]
axis_norm = (0,1)
if n_channel > 1:
    print("Normalizing image channels %s." % ('jointly' if axis_norm is None or 2 in axis_norm else 'independently'))

Normalizing image channels independently.


In [25]:
model = StarDist2D(None, name='stardist2D_100steps_400epochs_SplitScriptPatches_modelpatches256x256', basedir='models')
save_path = 'E:/Lempradl_Lab/April/OIC-78_Drosophila-brain-lesion-quant/StarDist_Model_Results'
StarDist2D_with_stitching(imgs=Ximgs,axis_norm=axis_norm,model=model,save_path=save_path)

Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.537479, nms_thresh=0.3.
Normalizing images


100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [01:41<00:00,  6.36s/it]


Predicting


  6%|█████▏                                                                             | 1/16 [01:00<15:02, 60.17s/it]functional.py (238): The structure of `inputs` doesn't match the expected structure.
Expected: ['input']
Received: inputs=Tensor(shape=(1, 3792, None, 1))
 12%|██████████▍                                                                        | 2/16 [02:00<14:06, 60.48s/it]functional.py (238): The structure of `inputs` doesn't match the expected structure.
Expected: ['input']
Received: inputs=Tensor(shape=(1, None, None, 1))
100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [13:45<00:00, 51.61s/it]


In [ ]:
norm = normalize(test_img, 1,99.8, axis=(0,)+tuple(1+np.array(axis_norm)))

In [ ]:
normX = []
for x in X:
    norm = normalize(x, 1,99.8, axis=(0,)+tuple(1+np.array(axis_norm)))
    normX.append(norm)

In [ ]:
labels = [model.predict_instances(frame, return_labels=True)[0] for frame in tqdm(norm)]

In [ ]:
labels = np.asarray(labels > 0 )

In [ ]:
labels = sk.measure.label(labels)

In [ ]:
test_img.shape

In [ ]:
help(model.predict_instances)

In [ ]:
#import napari
viewer = napari.view_image(test_img, name='img', scale=(3.0,0.2,0.2))
viewer.add_image(labels, name='labels',scale=(3.0,0.2,0.2))